In [155]:
!pip install newsapi-python
!pip install pyspark

In [156]:
pip install pandas

In [157]:
from pyspark.sql import SparkSession
from newsapi import NewsApiClient
from datetime import datetime, timedelta
import time
from pyspark.sql.functions import *
from pyspark.sql.functions import split


In [158]:
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('Iniciando com Spark') \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

In [159]:
##Mudem para a api_key de vcs please, se não vou ficar sem query rapidinho
##não se esqueça!!!!
API_KEY = '2d9f239e3c0d477c9726f8b48a20ecaa'

# Init
newsapi = NewsApiClient(api_key= API_KEY)

from_date = datetime.now() -  timedelta(days = 30)

WORDS = {
    'DNA': {'key_word': 'DNA', 'language': 'en'},
    'Ansiedade': {'key_word': 'anxiety', 'language': 'en'},
    'ELA': {'key_word': 'ELA', 'language': 'en'},

}


def fetch_news_data(key_word, language):
  params = {
        'q': key_word,
        'language': 'en',
        'from_param': from_date,
    }
  try:
    all_articles = newsapi.get_everything(q=key_word, language=language,from_param =from_date)
  except:
    return None

  if not all_articles['articles']:
    return

  news_df = spark.createDataFrame(all_articles['articles'])
  return news_df

def process_and_save_news(news_dataframe, key_word):
  news_df = news_dataframe
  news_df.dropDuplicates(['url'])
  news_df.write.mode("append").parquet(f"{key_word}_news.parquet")

  print(' --- SALVANDO  ---')



def update_news():
  while True:
    for words in WORDS.values():
      data = fetch_news_data(words['key_word'], words['language'])
      if data:
        process_and_save_news(data, words['key_word'])
    time.sleep(300)
    print(' ---ATUALIZANDO--- ')




In [160]:
update_news()


 --- SALVANDO  ---
 --- SALVANDO  ---
 --- SALVANDO  ---


KeyboardInterrupt: ignored

In [ ]:
arquivo = spark.read.parquet('/content/DNA_news.parquet')


In [ ]:
arquivo = arquivo.dropDuplicates(['url'])

In [ ]:
arquivo.count()

100

In [ ]:
arquivo = arquivo.withColumn('source',col('source.name'))

In [ ]:
arquivo.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|              author|             content|         description|         publishedAt|              source|               title|                 url|          urlToImage|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      Liliana Segura|Charlotte Beattie...|A college balleri...|2023-09-18T11:00:00Z|       The Intercept|DNA Evidence Sent...|http://theinterce...|https://theinterc...|
|GERALD IMRAY Asso...|CAPE TOWN, South ...|The South African...|2023-09-05T12:33:02Z|            ABC News|South Africa anno...|https://abcnews.g...|https://s.abcnews...|
|SEAN MURPHY Assoc...|McALESTER, Okla. ...|A man convicted o...|2023-09-21T16:21:55Z|            ABC News|Man executed for ...|https://abcnews.g...|ht

In [ ]:
arquivo.where(arquivo.author=='David Pescovitz').show()

+---------------+--------------------+--------------------+--------------------+-----------+--------------------+--------------------+--------------------+
|         author|             content|         description|         publishedAt|     source|               title|                 url|          urlToImage|
+---------------+--------------------+--------------------+--------------------+-----------+--------------------+--------------------+--------------------+
|David Pescovitz|An NOAA Ocean Exp...|An NOAA Ocean Exp...|2023-09-07T19:57:09Z|Boing Boing|Bizarre golden eg...|https://boingboin...|https://i0.wp.com...|
|David Pescovitz|Researchers at Ch...|Researchers at Ch...|2023-09-21T20:12:18Z|Boing Boing|Mutant worms engi...|https://boingboin...|https://i0.wp.com...|
+---------------+--------------------+--------------------+--------------------+-----------+--------------------+--------------------+--------------------+



In [ ]:
arquivo = arquivo.withColumn('publishedAt', split(col('publishedAt'),'-'))

In [ ]:
arquivo = arquivo.withColumn('year', col('publishedAt')[0])
arquivo = arquivo.withColumn('month', col('publishedAt')[1])
arquivo = arquivo.withColumn('day', col('publishedAt')[2])
arquivo.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----+-----+------------+
|              author|             content|         description|         publishedAt|              source|               title|                 url|          urlToImage|year|month|         day|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----+-----+------------+
|      Liliana Segura|Charlotte Beattie...|A college balleri...|[2023, 09, 18T11:...|       The Intercept|DNA Evidence Sent...|http://theinterce...|https://theinterc...|2023|   09|18T11:00:00Z|
|GERALD IMRAY Asso...|CAPE TOWN, South ...|The South African...|[2023, 09, 05T12:...|            ABC News|South Africa anno...|https://abcnews.g...|https://s.abcnews...|2023|   09|05T12:33:02Z|
|SEAN MURPHY Assoc...|McALESTE

In [ ]:
arquivo = arquivo.withColumn('day', split(col('day'),'T'))

In [ ]:
#arquivo = arquivo.withColumn('time', col('day')[1])
arquivo = arquivo.withColumn('day', col('day')[0])
arquivo = arquivo.drop('publishedAT').select(['title','author','day','month','year','content','description','source','url','urlToImage'])
arquivo.show()

+--------------------+--------------------+---+-----+----+--------------------+--------------------+--------------------+--------------------+--------------------+
|               title|              author|day|month|year|             content|         description|              source|                 url|          urlToImage|
+--------------------+--------------------+---+-----+----+--------------------+--------------------+--------------------+--------------------+--------------------+
|DNA Evidence Sent...|      Liliana Segura| 18|   09|2023|Charlotte Beattie...|A college balleri...|       The Intercept|http://theinterce...|https://theinterc...|
|South Africa anno...|GERALD IMRAY Asso...| 05|   09|2023|CAPE TOWN, South ...|The South African...|            ABC News|https://abcnews.g...|https://s.abcnews...|
|Man executed for ...|SEAN MURPHY Assoc...| 21|   09|2023|McALESTER, Okla. ...|A man convicted o...|            ABC News|https://abcnews.g...|https://s.abcnews...|
|Oregon man who 

In [ ]:
arquivo.where(arquivo.day=='07').show()


+--------------------+--------------------+---+-----+----+--------------------+--------------------+----------------+--------------------+--------------------+
|               title|              author|day|month|year|             content|         description|          source|                 url|          urlToImage|
+--------------------+--------------------+---+-----+----+--------------------+--------------------+----------------+--------------------+--------------------+
|Oregon man who wa...|ANDREW SELSKY Ass...| 07|   09|2023|SALEM, Ore. -- A ...|A man sentenced t...|        ABC News|https://abcnews.g...|https://i.abcnews...|
|Woman charged wit...|The Associated Press| 07|   09|2023|MORRISTOWN, N.J. ...|The mother of a n...|        ABC News|https://abcnews.g...|https://s.abcnews...|
|Bizarre golden eg...|     David Pescovitz| 07|   09|2023|An NOAA Ocean Exp...|An NOAA Ocean Exp...|     Boing Boing|https://boingboin...|https://i0.wp.com...|
|DNA analyses show...|           Science

In [ ]:
from collections import Counter

In [ ]:
day = arquivo.groupBy('day').agg(count(col('url')).alias('Contagem'))
month = arquivo.groupBy('month').agg(count(col('url')).alias('Contagem'))
author = arquivo.groupBy('author').agg(count(col('url')).alias('Contagem'))
source = arquivo.groupBy('source').agg(count(col('url')).alias('Contagem'))

In [ ]:
day.sort(['Contagem'],ascending = False).show(5)

+---+--------+
|day|Contagem|
+---+--------+
| 07|       9|
| 13|       9|
| 27|       8|
| 19|       7|
| 02|       7|
+---+--------+
only showing top 5 rows



In [ ]:
month.sort(['Contagem'],ascending = False).show(5)

+-----+--------+
|month|Contagem|
+-----+--------+
|   09|      89|
|   10|      10|
|   01|       1|
+-----+--------+



In [ ]:
author.sort(['Contagem'],ascending = False).show(5)

+--------------------+--------+
|              author|Contagem|
+--------------------+--------+
|                NULL|      19|
|           Science X|      10|
|The Associated Press|       3|
|https://www.faceb...|       3|
|    Andrew Tarantola|       2|
+--------------------+--------+
only showing top 5 rows



In [ ]:
source.sort(['Contagem'],ascending = False).show(5)

+----------------+--------+
|          source|Contagem|
+----------------+--------+
|        Phys.Org|      19|
|   Science Daily|      12|
|        BBC News|       6|
|        ABC News|       5|
|Business Insider|       4|
+----------------+--------+
only showing top 5 rows



In [ ]:
import pandas as pd

In [ ]:
arquivo_pd = arquivo.toPandas()
arquivo_pd.head(5)

,title,author,day,month,year,content,description,source,url,urlToImage
0,DNA Evidence Sent Anthony Sanchez to Death Row...,Liliana Segura,18,09,2023,Charlotte Beattie couldnt\r\n say when she beg...,A college ballerina was raped and murdered in ...,The Intercept,http://theintercept.com/2023/09/18/oklahoma-ex...,https://theintercept.com/wp-content/uploads/20...
1,South Africa announces official inquiry into d...,GERALD IMRAY Associated Press,05,09,2023,"CAPE TOWN, South Africa -- A retired Constitut...",The South African government says a retired ju...,ABC News,https://abcnews.go.com/International/wireStory...,https://s.abcnews.com/images/International/wir...
2,Man executed for the 1996 killing of a Univers...,SEAN MURPHY Associated Press,21,09,2023,"McALESTER, Okla. -- Oklahoma executed an inmat...",A man convicted of the 1996 killing of a Unive...,ABC News,https://abcnews.go.com/US/wireStory/man-set-ex...,https://s.abcnews.com/images/US/abc_news_defau...
3,Oregon man who was sentenced to death is free ...,ANDREW SELSKY Associated Press,07,09,2023,"SALEM, Ore. -- A man sentenced to death for a ...",A man sentenced to death for a 1998 murder is ...,ABC News,https://abcnews.go.com/US/wireStory/oregon-man...,https://i.abcnewsfe.com/a/3f18ab0e-36b3-4251-a...
4,Remains identified of airman who died in crash...,The Associated Press,08,09,2023,"MARQUETTE, Mich. -- Military scientists have i...",Military scientists have identified the remain...,ABC News,https://abcnews.go.com/US/wireStory/remains-id...,https://s.abcnews.com/images/US/abc_news_defau...


In [ ]:
colunas = list(arquivo_pd.keys())

['title',
 'author',
 'day',
 'month',
 'year',
 'content',
 'description',
 'source',
 'url',
 'urlToImage']

In [161]:
dna = ['dna','Dna','DNA']
contagem_palavras = {'Palavra': 'DNA', 'Contagem': []}
total = 0
for coluna in colunas:
  for palavra in dna:
    total += arquivo_pd[coluna].str.count(palavra).sum()
    contagem_palavras['Contagem'].append(arquivo_pd[coluna].str.count(palavra).sum())

print(f'total de palavras foi {total}')
print(contagem_palavras)

total de palavras foi 120.0
{'Palavra': 'DNA', 'Contagem': [0, 0, 27, 0.0, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 29, 0, 0, 40, 0, 0, 0, 17, 0, 1, 5.0, 0.0, 1.0]}
